<a href="https://colab.research.google.com/github/joseogg/textanalytics/blob/main/comparar_similitud_libros_resuelto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerías 

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt

# Corpus etiquetado

In [ ]:
etiquetas = ["shakespeare_the_merchant_of_venice", 
             "shakespeare_romeo_juliet", 
             "shakespeare_hamlet", 
             "dickens_a_christmas_carol", 
             "dickens_oliver_twist",
             "dickens_a_tale_of_two_cities",
             "melville_mobydick"]
corpus = []

for etiqueta in etiquetas:
  archivo = open(etiqueta + ".txt", "r")
  corpus.append(archivo.read())
  archivo.close()

etiquetas = ["shakespeare_the_merchant_of_venice", 
             "shakespeare_romeo_juliet", 
             "shakespeare_hamlet", 
             "dickens_a_christmas_carol", 
             "dickens_oliver_twist",
             "dickens_a_tale_of_two_cities",
             "melville_mobydick"]

corpus = np.array(corpus)
df_corpus = pd.DataFrame({"documento": corpus, 
                          "categoria": etiquetas})
df_corpus

,documento,categoria
0,"project,gutenberg,ebook,merchant,venice,willia...",shakespeare_the_merchant_of_venice
1,"project,gutenberg,ebook,romeo,juliet,william,s...",shakespeare_romeo_juliet
2,"project,gutenberg,ebook,hamlet,william,shakesp...",shakespeare_hamlet
3,"project,gutenberg,ebook,christmas,carol,charle...",dickens_a_christmas_carol
4,"project,gutenberg,ebook,oliver,twist,charles,d...",dickens_oliver_twist
5,"project,gutenberg,ebook,tale,two,city,charles,...",dickens_a_tale_of_two_cities
6,"project,gutenberg,ebook,moby,dick,whale,herman...",melville_mobydick


# Modelo de Bolsa de Palabras

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# bolsa de palabras en matriz dispersa
count_vectorizer = CountVectorizer(min_df=0.0, max_df=1.0)
matriz_conteo = count_vectorizer.fit_transform(corpus)
matriz_conteo

<7x20902 sparse matrix of type '<class 'numpy.int64'>'
	with 43754 stored elements in Compressed Sparse Row format>

In [ ]:
# ver valores diferentes de cero en la matriz dispersa
print(matriz_conteo)

  (0, 14158)	90
  (0, 8237)	99
  (0, 5861)	20
  (0, 11483)	20
  (0, 19745)	45
  (0, 20532)	4
  (0, 16085)	5
  (0, 2240)	8
  (0, 11244)	17
  (0, 15263)	18
  (0, 60)	2
  (0, 10591)	36
  (0, 18105)	12
  (0, 726)	5
  (0, 10237)	1
  (0, 2429)	11
  (0, 12882)	5
  (0, 1431)	1
  (0, 6274)	1
  (0, 8958)	1
  (0, 20505)	1
  (0, 231)	1
  (0, 18433)	8
  (0, 119)	7
  (0, 2324)	6
  :	:
  (6, 19474)	1
  (6, 19014)	1
  (6, 3604)	1
  (6, 18996)	1
  (6, 20021)	1
  (6, 2361)	1
  (6, 7667)	1
  (6, 6791)	1
  (6, 11854)	1
  (6, 10907)	1
  (6, 16953)	1
  (6, 20385)	1
  (6, 9806)	1
  (6, 3552)	1
  (6, 1494)	1
  (6, 17480)	1
  (6, 17963)	1
  (6, 6374)	1
  (6, 8292)	1
  (6, 14654)	1
  (6, 10029)	1
  (6, 10693)	1
  (6, 5209)	1
  (6, 19200)	1
  (6, 15149)	1


In [ ]:
# ver la representación densa
matriz_conteo = matriz_conteo.toarray()
matriz_conteo

array([[ 1,  0,  0, ...,  0,  0,  0],
       [ 3,  0,  0, ...,  2,  0,  0],
       [ 3,  0,  1, ...,  0,  0,  0],
       ...,
       [ 1,  0,  0, ...,  0,  0,  0],
       [ 1,  0,  0, ...,  0,  0,  0],
       [22,  0,  0, ...,  0,  1,  1]])

In [ ]:
# obten todas las palabras únicas del corpus
vocabulario = count_vectorizer.get_feature_names_out()
# muestra los vectores de características del documento
pd.DataFrame(matriz_conteo, columns=vocabulario)

,000,00033,04,08,0em,10,100,101,102,103,...,zig,zip,zodiac,zogranda,zone,zoology,zoroaster,zounds,ϰητος,חו
0,1,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,0,0,1,0,2,1,0,0,0,...,0,1,0,0,0,0,0,2,0,0
2,3,0,1,0,0,2,3,0,0,0,...,0,2,0,0,1,0,0,0,0,0
3,1,1,0,0,0,2,2,1,1,1,...,0,1,0,0,0,0,0,0,0,0
4,1,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,2,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
6,22,0,0,0,0,10,6,2,2,2,...,1,1,5,1,10,2,1,0,1,1


# N-gramas

In [ ]:
# Ajusta el rango del n-grama entre 1 y 2 to para obtener tanto unigramas como bigramas
count_vectorizer = CountVectorizer(ngram_range=(3, 3))
matriz_n_gramas = count_vectorizer.fit_transform(corpus)

matriz_n_gramas = matriz_n_gramas.toarray()
vocabulario = count_vectorizer.get_feature_names_out()
df = pd.DataFrame(matriz_n_gramas, columns=vocabulario)
df.T[df.sum()>30].sum(axis=1).sort_values(ascending=False)

project gutenberg electronic    72
say mr lorry                    72
gutenberg literary archive      65
literary archive foundation     65
project gutenberg literary      65
project gutenberg tm            65
say mr bumble                   65
gutenberg electronic work       64
say mr brownlow                 61
project gutenberg license       47
www gutenberg org               45
say miss pross                  44
project gutenberg work          40
full project gutenberg          35
project gutenberg ebook         35
ha ha ha                        32
dtype: int64

# Modelo TF-IDF 
* TF: term-frequency 
* TF-IDF: (term-frequency) * (inverse document-frequency)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

# Este par de líneas se ejecutan solo para obtener el vocabulario
count_vectorizer = CountVectorizer(min_df=0.0, max_df=1.0)
count_vectorizer.fit_transform(corpus)

# Aquí inicia la transformación
transformador = TfidfTransformer()
matriz_transformada = transformador.fit_transform(matriz_conteo)
matriz_transformada = matriz_transformada.toarray()
vocabulario = count_vectorizer.get_feature_names_out()
pd.DataFrame(np.round(matriz_transformada, 2), columns=vocabulario)

,000,00033,04,08,0em,10,100,101,102,103,...,zig,zip,zodiac,zogranda,zone,zoology,zoroaster,zounds,ϰητος,חו
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Similaridad entre documentos

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

matriz_similaridad = cosine_similarity(matriz_transformada)
df_similaridad = pd.DataFrame(matriz_similaridad)
df_similaridad

,0,1,2,3,4,5,6
0,1.000000,0.319743,0.212790,0.210831,0.256332,0.289983,0.234248
1,0.319743,1.000000,0.249286,0.196258,0.246967,0.295654,0.254330
2,0.212790,0.249286,1.000000,0.149360,0.180438,0.220304,0.193078
3,0.210831,0.196258,0.149360,1.000000,0.356418,0.393324,0.282543
4,0.256332,0.246967,0.180438,0.356418,1.000000,0.607272,0.361116
5,0.289983,0.295654,0.220304,0.393324,0.607272,1.000000,0.426179
6,0.234248,0.254330,0.193078,0.282543,0.361116,0.426179,1.000000
